### 1. 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 2. selenium 설치 & 내 드라이브에 chromedriver 설치

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' # (최초 1회)
!pip install chromedriver-autoinstaller

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [993 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,257 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [966 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-up

In [ ]:
# selenium 설치 확인
!python --version

import selenium
print(selenium.__version__)

Python 3.10.12
4.12.0


### 3. 실행 준비

#### 3.1 라이브러리 임포트

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sys
from selenium.webdriver.common.keys import Keys
import urllib.request
import os
from urllib.request import urlretrieve

import time
import pandas as pd
import chromedriver_autoinstaller  # setup chrome options

In [ ]:
chrome_path = "/content/drive/MyDrive/Colab Notebooks/chromedriver"

#### 3.1 chrome_options 설정

In [ ]:
sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off : cloab은 새창을 지원하지않기 때문에 창 없는 모드
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어

chromedriver_autoinstaller.install()  # set the target URL


#### 3.3 자동 스크롤 함수

In [ ]:
def selenium_scroll_option():
  SCROLL_PAUSE_SEC = 3

  # 스크롤 높이 가져옴
  last_height = driver.execute_script("return document.body.scrollHeight")

  while True:
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 10초 대기
    time.sleep(SCROLL_PAUSE_SEC)

    # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break
    last_height = new_height

### 4. 이미지 크롤링

In [ ]:
url = "https://www.google.com/"  # set up the webdriver

driver = webdriver.Chrome(options=chrome_options)

In [ ]:
# driver 작동 테스트
driver.get(url)
driver.implicitly_wait(3) # element가 로드될 때까지 지정한 시간만큼 대기할 수 있도록 설정
# driver.get_screenshot_as_file('google_screen.png')
driver.close()

print(driver)

<selenium.webdriver.chrome.webdriver.WebDriver (session="6eb13131fce409b10f441f82f350d373")>


In [ ]:
courbet = "/content/drive/MyDrive/Colab Notebooks/"

In [ ]:
# 키워드 검색하기

base_url = "https://www.google.co.kr/imghp?hl=ko" # 구글 이미지 검색
keyword=input("검색할 키워드를 입력 : ")
# print(type(a))
image_name = input("저장할 이미지 이름 : ")


driver = webdriver.Chrome(options=chrome_options)
driver.get(base_url)

# driver.find_element("xpath",'//*[@id="APjFqb"]') # 검색창 /html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/textarea
# browser = driver.find_element(By.ID, "APjFqb")

browser =  driver.find_element(By.NAME, "q")
browser.clear()
browser.send_keys(keyword)
browser.send_keys(Keys.RETURN)


# 클래스를 찾고 해당 클래스의 src 리스트를 만들자
selenium_scroll_option() # 스크롤하여 이미지를 많이 확보

'''이미지 src요소를 리스트업해서 이미지 url 저장'''

images = driver.find_elements(By.CSS_SELECTOR,".rg_i.Q4LuWd") #  클래스 네임에서 공백은 .을 찍어줌
images_url = []
for i in images:
    if i.get_attribute('src')!= None :
        images_url.append(i.get_attribute('src'))
    else :
        images_url.append(i.get_attribute('data-src'))
driver.close()



# 겹치는 이미지 url 제거

print("전체 다운로드한 이미지 개수: {}\n동일한 이미지를 제거한 이미지 개수: {}".format(len(images_url), len(pd.DataFrame(images_url)[0].unique())))
images_url=pd.DataFrame(images_url)[0].unique()

if image_name == 'courbet' :
    for t, url in enumerate(images_url, 0):
        urlretrieve(url, courbet + image_name + '_' + str(t) + '.jpg')
    # driver.close()


검색할 키워드를 입력 : courbet
저장할 이미지 이름 : courbet
전체 다운로드한 이미지 개수: 400
동일한 이미지를 제거한 이미지 개수: 397


In [ ]:
# 자원회수 - 코드 실행 마지막에 quit driver
driver.quit()